In [2]:
!pip install --quiet --upgrade gradio
!pip install --quiet openai
!pip install --quiet astrapy
!pip install --quiet voyageai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 5.7 MB/s 

In [3]:
#Setting up useful libraries
from openai import OpenAI
import sys
import codecs
import os
import pandas as pd
from tqdm import tqdm
from google.colab import userdata
import voyageai
from astrapy.db import AstraDB
import gradio as gr

vec_db = AstraDB(
  token="AstraCS:JojuxKENIQUZbHOHsOKzqdxZ:311ac2f4ac1f981f7c69a4bb8566b0bcac48193257fa845da7cff3aabb38395f",
  api_endpoint="https://8b4697c6-4dca-40ac-8a07-7f098c39314b-us-east1.apps.astra.datastax.com",# This has the dataset
  namespace = 'student_services')

print(f"Connected to Astra DB: {vec_db.get_collections()}")

vstore = vec_db.create_collection('StudyBuddy', dimension=1536, metric='cosine')

vo = voyageai.Client(api_key= '*********************************')# Add your voyage ai api key here

# Set the collection name
collection = vec_db.collection(collection_name="StudyBuddy")

client = OpenAI(
      base_url="https://api-inference.huggingface.co/v1",
      api_key = '***************************************',# Add your huggingface api key here
  )

client_open_ai = OpenAI(api_key = '*****************************************')# Add your openai api key here

Connected to Astra DB: {'status': {'collections': ['StudyBuddy']}}


In [4]:

recommendations = None
possible_matches = None
student_details = None
student_name = None


with gr.Blocks(
    title = 'Expatrio Matching Algorithm'
) as demo:
    gr.Markdown("Answer the following few questions to build your query. Feel free to leave blank or put N/A for questions that you prefer not to answer / do not apply. You can also add any additional preferences or information in the last question.")
    name = gr.Textbox(label="What is your name?")
    degree = gr.Textbox(label="What degree are you currently seeking (Masters, PhD etc)?")
    major = gr.Textbox(label="Could you share your intended major?")
    tuition = gr.Textbox(label="Are you working within a preferred tuition budget?")
    style_of_learning = gr.Textbox(label="Would you say you learn more towards hands-on, experiential learning where you actively engage in practical activities and projects, or do you feel more comfortable with traditional classroom-based learning, focusing on lectures and discussions?")
    location = gr.Textbox(label="Do you have a preference for the location of your program within Germany?")
    top_priority = gr.Textbox(label="Among academics, professional opportunities, social environment, and cost of living, which factor holds the greatest importance for you when selecting a university?")
    social_life_importance = gr.Textbox(label="On a scale of 1 to 5, to what extent do you value maintaining an active social life and participating in extracurricular activities beyond your academic studies?")
    financial_aid_relevance = gr.Textbox(label="Could financial aid or scholarship opportunities play a significant role in your decision to enroll in a university?")
    cohort_size = gr.Textbox(label="When it comes to class sizes, do you prefer smaller cohorts for a more personalized learning experience, or do you thrive in larger academic communities with a diverse range of perspectives?")
    inclusion_diversity = gr.Textbox(label="To what extent do you value attending a university that places a strong emphasis on inclusion and diversity among its student body and faculty?")
    projects = gr.Textbox(label="Are you particular about participating in research projects, internships, or other hands-on learning opportunities during your undergraduate studies?")
    student_clubs = gr.Textbox(label="Are you particular on joining any particular student clubs or organizations on campus?")
    reputation = gr.Textbox(label="How much importance do you place on the reputation and prestige of a university when deciding where to study?")
    preferences = gr.Textbox(label="Do you have any peculiar preferences not listed above?")
    submit_btn = gr.Button("Submit", variant="primary")
    clear_btn = gr.ClearButton(value = "Reset All", components=[name, degree, major, tuition, style_of_learning, location, top_priority, social_life_importance, financial_aid_relevance, cohort_size, inclusion_diversity, projects, student_clubs, reputation, preferences])
    output= gr.Textbox(label="This is the summary of your data. Feel free to adjust any information you'd like to change.", interactive=True)
    all_good_btn = gr.Button("This looks good", variant="secondary")

    @submit_btn.click(inputs=[name, degree, major, tuition, style_of_learning, location, top_priority, social_life_importance, financial_aid_relevance, cohort_size, inclusion_diversity, projects, student_clubs, reputation, preferences], outputs=output)

    def build_query(name, degree, major, tuition, style_of_learning, location, top_priority, social_life_importance, financial_aid_relevance, cohort_size, inclusion_diversity, reputation, projects, student_clubs, preferences):
        query = f"""
        My name is {name}. I am seeking a {degree} course with {major} major. My tuition fee constraints are {tuition},
        {style_of_learning} is my preferred style of learning, {location} is my preferred location for the program.
        My topmost priority in choosing a program is one with a good {top_priority}. On a scale of 1 to 5, {social_life_importance} is my preference for a program with strong social activities and {financial_aid_relevance} is how important financial aid is to me.
        My preferences for the program's cohort size is {cohort_size}, inclusion and diversity is {inclusion_diversity}, reputation is {reputation}.
        I would love a program with the opportunity to engage in {projects} and join {student_clubs} student clubs. Finally, {preferences} are additional preferences to keep in mind."""

        global student_name
        student_name = name

        chat_completion = client.chat.completions.create(
          model = 'google/gemma-7b-it', #gpt-4
          messages = [
              {
                  "role": "user",
                  "content": f"""
                  Correct the provided query {query} for a user who is looking for a program that matches their needs to make it more coherent.
                  Provide only the rewritten query starting with "My name is" without additional remarks. If some spaces in the query are blank, leave them blank. Do not hallucinate information to fill them in.
                  """
              }
          ],
          max_tokens=5000
        )

        global student_details
        student_details = chat_completion.choices[0].message.content

        return student_details

    output.input(outputs=output)

    #print(output)
    recommendationsTextbox = gr.Textbox(label="Based on your query, here are some recommendations for you. Feel free to ask further questions about the recommended programs in the chatbot below.")

    @all_good_btn.click(inputs=[output], outputs=[recommendationsTextbox])

    def get_chat_matches(query):
        def get_matching_embeddings(query):
            embeddings = vo.embed(query, model='voyage-large-2', input_type='query').embeddings[0]
            max_records = 3

            # Retrieve all documents based on the vector search
            result = collection.vector_find(
                vector=embeddings,
                limit=max_records
            )

            response = []

            for i in range(len(result)):
                response.append(f"{result[i]['content']}")
                #print(result[i]['content'])
                #print('\n')

            return response

        global possible_matches
        possible_matches = get_matching_embeddings(query)

        chat_completion = client.chat.completions.create(
                model = 'google/gemma-7b-it', #gpt-4
                messages = [
                    {
                        "role": "user",
                        "content": f""" You are a course adviser for a student looking for a course in Germany.
                        Given the student's query {student_details} and possible responses {possible_matches}
                        Start by saying 'Hey {student_name}, based on your interests and preferences, I found the following courses'.
                        To determine the best fit, prioritize the response that aligns most with {degree} and then {major}.
                        You may also draw on external, accurate knowledge about the suggested university/program to determine if it is a good fit
                        Present the best fit and highlight why each is a good fit for their query and which ways it doesn't fit the query perfectly
                        """
                    }
                ],
                max_tokens=5000
            )
        #Then present each of the possible responses to the the student highlighting why each is a good fit for their query
        global recommendations

        recommendations = chat_completion.choices[0].message.content
        return recommendations

    def predict(message, history):
      if not recommendations:
        yield 'Please fill out the form above first'

      else:
        history_openai_format = []

        for human, assistant in history:
          history_openai_format.append({"role": "user", "content": human })
          history_openai_format.append({"role": "assistant", "content": assistant})

        history_openai_format.append({"role": "user", "content": message})
        system_message = {
                "role": "system",
                "content": f"""
                def drawUponKnowledgeFromTrainingData(message):
                  You are a friendly expert on German universities and programs. Use your knowledge to respond to message.
                  return answerToMessage

                def giveResponseToMessage(student_details, possible_matches, recommendations, message):
                  '''
                  Given a message, respond to the student in an engaging way

                  Input
                  student_details: A summary of the students preferences in searching for a German program
                  possible_matches: Programs they have been matched with
                  recommendations: A summary of the possible matches and how they fit the students details
                  message: Student's additional question.

                  Output:
                  answerToMessage: Your response to the student's additional question.
                  '''

                  if answer to message not in student_details or possible_matches or recommendations:
                    respond from your general knowledge

                  else:
                    check (student_details and possible_matches and recommendations):
                      return answerToMessage.

                  def giveResponseToMessage({student_details, possible_matches, recommendations, message})
                """}

        #Engage conversationally with the student's message, {message} and draw upon your extensive knowledge of German universities and programs to help them where necessary.
        #Where necessary to respond, you may draw upon the background information about the student: {student_details}. They have been matched with the following possible programs: {possible_matches} and provided the following recommendations: {recommendations}.
        response = client_open_ai.chat.completions.create(model='gpt-3.5-turbo',
        messages= [system_message, {'role': 'user', 'content': message}],
        stream=False)

        #print(response.choices[0].message.content)
        yield response.choices[0].message.content
        '''
        responseString = ""
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                  responseString = responseString + chunk.choices[0].delta.content
        print(responseString)

        partial_message = ""
        for chunk in response:
            if chunk.choices[0].delta.content is not None:
                  partial_message = partial_message + chunk.choices[0].delta.content
                  yield partial_message
        '''

    gr.ChatInterface(predict)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9fc7bdc37595bb7629.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
